In [6]:
import os
import numpy as np
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.optimizers import Adam, Adamax
from keras import regularizers
from PIL import Image

In [7]:
train_file = "C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir"

In [8]:
filepaths = []
labels = []

In [9]:
def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()  # Verify image integrity
            return True
    except (IOError, SyntaxError) as e:
        print(f"Invalid image: {file_path} ({e})")
        return False

valid_images = []
for image_file in os.listdir(train_file):
    image_path = os.path.join(train_file, image_file)
    if is_valid_image(image_path):
        valid_images.append(image_path)

Invalid image: C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\cat_image_12500.db (cannot identify image file 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_12500.db')
Invalid image: C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\cat_image_8790.jpg (cannot identify image file 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_8790.jpg')


c:\Users\HomeLander\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Invalid image: C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\dog_image_12500.db (cannot identify image file 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\dog_image_12500.db')
Invalid image: C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\dog_image_1895.jpg (cannot identify image file 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\dog_image_1895.jpg')


In [10]:
valid_images

['C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_0.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_1.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_100.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_1000.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10000.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10001.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10002.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10003.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10004.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10005.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10006.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_10007.jpg',
 'C:/Users/HomeLander/Desktop/WorkSpace 2/new_dir\\cat_image_1000

In [11]:
for imagename in valid_images:
    group1 = imagename.split('\\')[1]
    group = group1.split("_")[0]
    
    if group == 'cat':
        labels.append('cat')
    else:
        labels.append('dog')

train_df = pd.DataFrame({
    'image_id': valid_images,
    'target': labels
})

In [12]:
train_df

,image_id,target
0,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,cat
1,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,cat
2,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,cat
3,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,cat
4,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,cat
...,...,...
24993,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,dog
24994,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,dog
24995,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,dog
24996,C:/Users/HomeLander/Desktop/WorkSpace 2/new_di...,dog


In [13]:
train_df, test_df = train_test_split(train_df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df .reset_index(drop=True)


train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    x_col='image_id',
    y_col='target',
    target_size=(64, 64),
    class_mode='categorical',
    batch_size= 15
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    test_df, 
    x_col='image_id',
    y_col='target',
    target_size=(64, 64),
    class_mode='categorical',
    batch_size= 15
)

Found 19998 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [14]:
CNN_Model = tf.keras.applications.efficientnet.EfficientNetB3(
    include_top = False, 
    weights = "imagenet", 
    input_shape = (64, 64, 3), 
    pooling = 'max'
)


model = Sequential([
    CNN_Model,
    BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001),
    Dense(units = 256, kernel_regularizer = regularizers.l2(l = 0.016), activity_regularizer = regularizers.l1(0.006), bias_regularizer = regularizers.l1(0.006), activation = 'relu'),
    Dropout(rate = 0.45, seed = 123),
    Dense(units = 2, activation = 'softmax')
])


model.compile(Adamax(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(
    x = train_generator, 
    epochs = 20, 
    verbose = 1, 
    validation_data = validation_generator,          
    validation_steps = None, 
    shuffle = False
)

43941136/43941136 [==============================] - 32s 1us/step
Epoch 1/20
1334/1334 [==============================] - ETA: 0s - loss: 3.0535 - accuracy: 0.7474

c:\Users\HomeLander\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1334/1334 [==============================] - 479s 347ms/step - loss: 3.0535 - accuracy: 0.7474 - val_loss: 1.0294 - val_accuracy: 0.8166
Epoch 2/20
1334/1334 [==============================] - 480s 360ms/step - loss: 0.6890 - accuracy: 0.8341 - val_loss: 0.6677 - val_accuracy: 0.7074
Epoch 3/20
1334/1334 [==============================] - 482s 361ms/step - loss: 0.4367 - accuracy: 0.8690 - val_loss: 0.8789 - val_accuracy: 0.6112
Epoch 4/20
1334/1334 [==============================] - 482s 361ms/step - loss: 0.3727 - accuracy: 0.8788 - val_loss: 0.4532 - val_accuracy: 0.8344
Epoch 5/20
1334/1334 [==============================] - 483s 362ms/step - loss: 0.3257 - accuracy: 0.9039 - val_loss: 0.6973 - val_accuracy: 0.5510
Epoch 6/20
1334/1334 [==============================] - 480s 360ms/step - loss: 0.2936 - accuracy: 0.9150 - val_loss: 0.5077 - val_accuracy: 0.7858
Epoch 7/20
1334/1334 [==============================] - 473s 355ms/step - loss: 0.2706 - accuracy: 0.9243 - val_loss: 0.277

In [ ]:
CNN = tf.keras.applications.convnext.ConvNeXtLarge(
    model_name='convnext_large',
    include_top = False, 
    weights = "imagenet", 
    input_shape = (64, 64, 3), 
    pooling = 'max'
)
model_CNN = Sequential([
    CNN,
    BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001),
    Dense(units = 256, kernel_regularizer = regularizers.l2(l = 0.016), activity_regularizer = regularizers.l1(0.006), bias_regularizer = regularizers.l1(0.006), activation = 'relu'),
    Dropout(rate = 0.45, seed = 123),
    Dense(units = 2, activation = 'softmax')
])


model_CNN.compile(Adamax(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model_CNN.fit(
    x = train_generator, 
    epochs = 20, 
    verbose = 1, 
    validation_data = validation_generator,          
    validation_steps = None, 
    shuffle = False
)

In [ ]:
CModel = tf.keras.applications.densenet.DenseNet169(
    include_top = False, 
    weights = "imagenet", 
    input_shape = (224, 224, 3), 
    pooling = 'max'
)

modelCNN = Sequential([
    CModel,
    BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001),
    Dense(units = 256, kernel_regularizer = regularizers.l2(l = 0.016), activity_regularizer = regularizers.l1(0.006), bias_regularizer = regularizers.l1(0.006), activation = 'relu'),
    Dropout(rate = 0.45, seed = 123),
    Dense(units = 2, activation = 'softmax')
])


modelCNN.compile(Adamax(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = modelCNN.fit(
    x = train_generator, 
    epochs = 20, 
    verbose = 1, 
    validation_data = validation_generator,          
    validation_steps = None, 
    shuffle = False
)